In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns",25)
pd.set_option("display.max_rows",250)
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from urllib.request import urlopen
import json

In [2]:
path = 'C:/Users/marcusdeckert/Box/marcusdeckert/Presentation Work/CARFAX Academy/data/'
data5 = pd.read_feather(path + 'data5.ftr')

In [3]:
[i for i in data5.columns]

['pol_eff_year',
 'ep_bi',
 'ep_col',
 'ee_bi',
 'ee_col',
 'incloss_bi',
 'incloss_col',
 'cc_bi',
 'cc_col',
 'zip',
 'pol_id',
 'vin_id',
 'credit',
 'commute_flag',
 'veh_count_box',
 'driver_count_box',
 'veh_age_box',
 'coll_symb_ntile',
 'limit_bi',
 'ded_coll',
 'DrvAge_box',
 'male',
 'single',
 'widowed',
 'Date',
 'Modeled_Annual_Mileage',
 'Estimated_Current_Mileage',
 'Annual_Mileage_Estimate',
 'Number_of_Titling_Transactions',
 'Lien_Holder',
 'current_ownership_personal',
 'current_ownership_lease',
 'current_ownership_corp_govt',
 'LOO_years',
 'personal_use_flag',
 'rental_use_flag',
 'comm_use_flag',
 'fleet_use_flag',
 'corp_use_flag',
 'lease_flag',
 'curr_owner_odo_cnt',
 'all_owner_odo_cnt',
 'Severe_Problem_flag',
 'Branded_Title_flag',
 'Branded_Title_Loss_flag',
 'Severe_Accident_flag',
 'Other_Severe_Problem_flag',
 'Failed_Emissions_flag',
 'Nonsevere_Accident_flag',
 'Damage_flag',
 'Collision_Repair_Facility_flag',
 'Potential_Damage_flag',
 'Odometer_Prob

In [4]:
### Function to make useful Collision stats
facts = ['ee_col','incloss_col','cc_col','ep_col']

def stats_by_zip(data):
    z = data.groupby(['zip']).agg({f: 'sum' for f in facts}).reset_index()
    
    z['incloss_col'] = np.where(z['incloss_col'] < 0, 0, z['incloss_col'])
    
    z['Exposure %'] = z['ee_col']/z['ee_col'].sum()
    z['Frequency'] = round(z['cc_col']/z['ee_col'],3)
    z['Severity'] = round(z['incloss_col']/z['cc_col'],0)
    z['Pure Premium'] = round(z['incloss_col']/z['ee_col'],0)
    z['Loss Ratio'] = round(z['incloss_col']/z['ep_col'],0)
    
    return z[(['zip','Exposure %','Frequency','Severity','Pure Premium','Loss Ratio'])]

In [5]:
stats_by_zip(data5)

,zip,Exposure %,Frequency,Severity,Pure Premium,Loss Ratio
0,43001,0.00,0.00,NaN,0.00,0.00
1,43003,0.00,0.00,NaN,0.00,0.00
2,43004,0.00,0.00,NaN,0.00,0.00
3,43006,0.00,0.01,"1,115.00",17.00,0.00
4,43008,0.00,0.00,NaN,0.00,0.00
...,...,...,...,...,...,...
1056,45891,0.00,0.00,NaN,0.00,0.00
1057,45894,0.00,0.04,"1,685.00",59.00,0.00
1058,45895,0.00,0.12,"2,738.00",342.00,4.00
1059,45896,0.00,0.02,"2,410.00",39.00,0.00


### Folium Map first try

In [8]:
path = 'C:/Users/marcusdeckert/Box/marcusdeckert/Presentation Work/CARFAX Academy/Lesson 5 stuff/'

import folium

def loss_ratio_folium_map(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str') #skipped accounting for zip codes with leading 0's because we are doing Ohio

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [1]:
loss_ratio_folium_map(data5)

In [10]:
# limit to p5 and p95
def loss_ratio_folium_map2(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str')
    
    ### SECTION ADDED
    # limit the range of values to p5 and p95
    
    min_val = zip_data['Loss Ratio'].quantile(q = .05)
    max_val = zip_data['Loss Ratio'].quantile(q = .95)
    
    zip_data['Loss Ratio Display'] = np.where(
        zip_data['Loss Ratio'] < min_val, min_val,
        np.where(
            zip_data['Loss Ratio'] > max_val, max_val, zip_data['Loss Ratio']
        )
    )
    
    ### END SECTION ADDED

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio Display'], ### CHANGED
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code; p5 to p95'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [2]:
loss_ratio_folium_map2(data5)

In [12]:
# change to [0,1]
def loss_ratio_folium_map3(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str')
    
    min_val = 0 ### CHANGED
    max_val = 1 ### CHANGED
    
    zip_data['Loss Ratio Display'] = np.where(
        zip_data['Loss Ratio'] < min_val, min_val,
        np.where(
            zip_data['Loss Ratio'] > max_val, max_val, zip_data['Loss Ratio']
        )
    )

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio Display'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code; Limit 100%'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [3]:
loss_ratio_folium_map3(data5)

### Add Hoverinfo

In [14]:
### First, let's look at the geojson again

with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
    zips_json = json.load(f)
    
zips_json['features'][0]

{'type': 'Feature',
 'properties': {'STATEFP10': '39',
  'ZCTA5CE10': '45830',
  'GEOID10': '3945830',
  'CLASSFP10': 'B5',
  'MTFCC10': 'G6350',
  'FUNCSTAT10': 'S',
  'ALAND10': 245664720,
  'AWATER10': 465444,
  'INTPTLAT10': '+40.9084596',
  'INTPTLON10': '-084.0959329',
  'PARTFLG10': 'N'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-84.051797, 40.846578],
     [-84.053422, 40.846572],
     [-84.058143, 40.846593],
     [-84.062935, 40.846592],
     [-84.065125, 40.846594],
     [-84.066055, 40.846587],
     [-84.066278, 40.846585],
     [-84.067145, 40.846579],
     [-84.068737, 40.846582],
     [-84.071053, 40.846575],
     [-84.071057, 40.846714],
     [-84.07106, 40.847188],
     [-84.071056, 40.848023],
     [-84.071066, 40.848851],
     [-84.071073, 40.849537],
     [-84.071075, 40.849683],
     [-84.07108, 40.850515],
     [-84.071076, 40.851096],
     [-84.071075, 40.85136],
     [-84.071084, 40.852199],
     [-84.071098, 40.853024],
     [-84.071094, 40.85

In [15]:
### We need the geojson to contain the needed information so it can be displayed as a "child"

# We want to have the pop up show Zip, Exposure %,Frequency,Severity,Pure Premium,Loss Ratio

# Let's begin by simply adding an element called 'Zip' to a copy of the geojson

geo_data = zips_json.copy()
for i in range(0,len(geo_data['features']) - 1): # consider refactoring these 2 lines to directly call each element
    geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON
    
geo_data['features'][0]

{'type': 'Feature',
 'properties': {'STATEFP10': '39',
  'ZCTA5CE10': '45830',
  'GEOID10': '3945830',
  'CLASSFP10': 'B5',
  'MTFCC10': 'G6350',
  'FUNCSTAT10': 'S',
  'ALAND10': 245664720,
  'AWATER10': 465444,
  'INTPTLAT10': '+40.9084596',
  'INTPTLON10': '-084.0959329',
  'PARTFLG10': 'N',
  'Zip': '45830'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-84.051797, 40.846578],
     [-84.053422, 40.846572],
     [-84.058143, 40.846593],
     [-84.062935, 40.846592],
     [-84.065125, 40.846594],
     [-84.066055, 40.846587],
     [-84.066278, 40.846585],
     [-84.067145, 40.846579],
     [-84.068737, 40.846582],
     [-84.071053, 40.846575],
     [-84.071057, 40.846714],
     [-84.07106, 40.847188],
     [-84.071056, 40.848023],
     [-84.071066, 40.848851],
     [-84.071073, 40.849537],
     [-84.071075, 40.849683],
     [-84.07108, 40.850515],
     [-84.071076, 40.851096],
     [-84.071075, 40.85136],
     [-84.071084, 40.852199],
     [-84.071098, 40.853024],
     

In [16]:
### We need the other statistics to show in hoverinfo from the dataframe
# pick this first zip and grab the values

data = stats_by_zip(data5)

data.loc[data['zip'] == 45830]

,zip,Exposure %,Frequency,Severity,Pure Premium,Loss Ratio
1012,45830,0.00,0.00,NaN,0.00,0.00


In [17]:
### Get just the Exposure %

data.loc[data['zip'] == 45830]['Exposure %'].sum()

0.00022093706202021632

In [18]:
# format it nicely

'{:.1%}'.format(data.loc[data['zip'] == 45830]['Exposure %'].sum())

'0.0%'

In [19]:
### Add the rest of the information now
data = stats_by_zip(data5)

geo_data = zips_json.copy()
for i in range(0,len(geo_data['features']) - 1): # if you redid the one above do this one, too
    geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON
    geo_data['features'][i]['properties']['Exposure %'] = 'Exposure %: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Exposure %'].sum())
    geo_data['features'][i]['properties']['Frequency'] = 'Frequency: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Frequency'].sum())
    geo_data['features'][i]['properties']['Severity'] = 'Severity: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Severity'].sum())
    geo_data['features'][i]['properties']['Pure Premium'] = 'Pure Premium: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Pure Premium'].sum())
    geo_data['features'][i]['properties']['Loss Ratio'] = 'Loss Ratio: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Loss Ratio'].sum())
    
geo_data['features'][0]

{'type': 'Feature',
 'properties': {'STATEFP10': '39',
  'ZCTA5CE10': '45830',
  'GEOID10': '3945830',
  'CLASSFP10': 'B5',
  'MTFCC10': 'G6350',
  'FUNCSTAT10': 'S',
  'ALAND10': 245664720,
  'AWATER10': 465444,
  'INTPTLAT10': '+40.9084596',
  'INTPTLON10': '-084.0959329',
  'PARTFLG10': 'N',
  'Zip': '45830',
  'Exposure %': 'Exposure %: 0.0%',
  'Frequency': 'Frequency: 0.0%',
  'Severity': 'Severity: 0.0',
  'Pure Premium': 'Pure Premium: 0.0',
  'Loss Ratio': 'Loss Ratio: 0.0%'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-84.051797, 40.846578],
     [-84.053422, 40.846572],
     [-84.058143, 40.846593],
     [-84.062935, 40.846592],
     [-84.065125, 40.846594],
     [-84.066055, 40.846587],
     [-84.066278, 40.846585],
     [-84.067145, 40.846579],
     [-84.068737, 40.846582],
     [-84.071053, 40.846575],
     [-84.071057, 40.846714],
     [-84.07106, 40.847188],
     [-84.071056, 40.848023],
     [-84.071066, 40.848851],
     [-84.071073, 40.849537],
     [-

In [ ]:
### Add this into the map function and add the child

In [20]:
def loss_ratio_folium_map4(data):
    data = stats_by_zip(data) ### Changed
    data['zip'] = data['zip'].astype('str') ### Changed
    
    min_val = 0
    max_val = 1
    
    ### Changed
    data['Loss Ratio Display'] = np.where(
        data['Loss Ratio'] < min_val, min_val,
        np.where(
            data['Loss Ratio'] > max_val, max_val, data['Loss Ratio']
        )
    )

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)
        
    ### Added
    geo_data = zips_json.copy()
    for i in range(0,len(geo_data['features']) - 1):
        geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON
        geo_data['features'][i]['properties']['Exposure %'] = 'Exposure %: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Exposure %'].sum())
        geo_data['features'][i]['properties']['Frequency'] = 'Frequency: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Frequency'].sum())
        geo_data['features'][i]['properties']['Severity'] = 'Severity: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Severity'].sum())
        geo_data['features'][i]['properties']['Pure Premium'] = 'Pure Premium: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Pure Premium'].sum())
        geo_data['features'][i]['properties']['Loss Ratio'] = 'Loss Ratio: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Loss Ratio'].sum())
    ### END Added

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = geo_data, ### Changed
            data = data, ### Changed
            columns = ['zip','Loss Ratio Display'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )
    
    ### Added 
    for key in choro._children:
        if key.startswith('color_map'):
            del(choro._children[key])
    ### END Added

    choro.add_to(m)
    
    ### Added
    folium.LayerControl().add_to(m)
    choro.geojson.add_child(
        folium.features.GeoJsonTooltip(['Zip','Exposure %','Frequency','Severity','Pure Premium','Loss Ratio'], labels=False)
    )
    ### END Added

    # Title
    title = 'Loss Ratio by Zip Code; Limit 100%'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [4]:
loss_ratio_folium_map4(data5)

In [22]:
with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
    zips_json = json.load(f)

geo_data = zips_json.copy()

x = geo_data['features'][0]

In [23]:
x2 = pd.DataFrame(x).T

In [24]:
x2

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,type,coordinates
type,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature
properties,39,45830,3945830,B5,G6350,S,245664720,465444,+40.9084596,-084.0959329,N,NaN,NaN
geometry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MultiPolygon,"[[[[-84.051797, 40.846578], [-84.053422, 40.84..."


In [25]:
zip_df = stats_by_zip(data5)
zip_df['zip'] = zip_df['zip'].astype('str')

x3 = x2.merge(zip_df, left_on = 'ZCTA5CE10', right_on = 'zip', how = 'left')

x3

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,type,coordinates,zip,Exposure %,Frequency,Severity,Pure Premium,Loss Ratio
0,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,Feature,NaN,NaN,NaN,NaN,NaN,NaN
1,39,45830,3945830,B5,G6350,S,245664720,465444,+40.9084596,-084.0959329,N,NaN,NaN,45830,0.00,0.00,NaN,0.00,0.00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MultiPolygon,"[[[[-84.051797, 40.846578], [-84.053422, 40.84...",NaN,NaN,NaN,NaN,NaN,NaN
